In [ ]:
import os
import glob
import math

import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import box
from geopy.distance import distance

from rasterio.transform import from_bounds
from rasterio.crs import CRS

### 1. Break Polygon into Grids

In [ ]:
df = gpd.read_file(r"../data/yerevan_boundary/yerevan.shp")

In [ ]:
df.set_crs(epsg=4326, inplace=True)

In [ ]:
# Dissolve all polygons into a single polygon

df['dummy'] = 1
df = df.dissolve(by='dummy')

In [9]:
# Step 1: Project to a CRS in meters
df_meters = df.to_crs(epsg=3857)  # Web Mercator (meters)

# Step 2: Apply a 500-meter buffer
df_buffered = df_meters.buffer(1500)

# Step 3: Optional - Convert back to original CRS (4326)
df = gpd.GeoDataFrame(geometry=df_buffered, crs=df_meters.crs).to_crs(epsg=4326)

In [12]:
def make_grid(gdf, cell_width_m, cell_height_m):
    bounds = gdf.total_bounds
    min_x, min_y, max_x, max_y = bounds
    width = max_x - min_x
    height = max_y - min_y
    
    # Get the latitude at the center of the bounding box
    center_lat = (min_y + max_y) / 2
    
    # Convert cell width and height from meters to degrees
    cell_width = cell_width_m / distance((center_lat, min_x), (center_lat, min_x + 1)).meters
    cell_height = cell_height_m / distance((min_y, center_lat), (min_y + 1, center_lat)).meters

    rows = int(math.ceil(height / cell_height))
    cols = int(math.ceil(width / cell_width))

    grid_cells = []
    for i in range(cols):
        for j in range(rows):
            x1 = min_x + i * cell_width
            y1 = min_y + j * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            grid_cells.append(box(x1, y1, x2, y2))
    
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=gdf.crs)
    return grid

# Define the cell width and height in meters
cell_width_m = 545  # Corrected width
cell_height_m = 305  # Height remains the same

# Create the grid
grid = make_grid(df, cell_width_m, cell_height_m)

# Clip the grid to the city boundary
grid_clipped = gpd.clip(grid, df)

In [ ]:
grid_clipped["Index"] = range(0,len(grid_clipped))

In [15]:
grid_clipped

,geometry,Index
2084,"POLYGON ((44.56442 40.06911, 44.56528 40.06911...",0
2085,"POLYGON ((44.56442 40.06911, 44.56442 40.07185...",1
2086,"POLYGON ((44.56442 40.07185, 44.56442 40.07460...",2
2087,"POLYGON ((44.56442 40.07460, 44.56442 40.07735...",3
2088,"POLYGON ((44.56442 40.07735, 44.56442 40.08009...",4
...,...,...
2238,"POLYGON ((44.57081 40.26961, 44.57081 40.27026...",2250
1918,"POLYGON ((44.55162 40.27236, 44.55162 40.26961...",2251
1999,"POLYGON ((44.55162 40.27236, 44.55162 40.27267...",2252
1919,"POLYGON ((44.55162 40.27236, 44.54889 40.27236...",2253


In [ ]:
# grid_clipped.to_file("../data/yerevan_grid.geojson",driver="GeoJSON")

### 2. Georeferencing Images

In [ ]:
grid = gpd.read_file("../data/yerevan_grid.geojson") 

In [ ]:
grid = grid.to_crs("EPSG:4326")

# Output directory
os.makedirs("../images_georef", exist_ok=True)

for idx, row in grid.iterrows():
    polygon = row.geometry
    index = row["Index"]
    
    # Use glob to find matching image file
    matching_images = glob.glob(f"images/{index}_*.jpg")
    
    if not matching_images:
        print(f"No image found for index {index}")
        continue
    
    image_path = matching_images[0]
    
    # Open image and get dimensions
    img = Image.open(image_path)
    width, height = img.size

    # Get bounds from polygon
    minx, miny, maxx, maxy = polygon.bounds

    # Calculate transform
    transform = from_bounds(minx, miny, maxx, maxy, width, height)

    # Prepare output path
    output_path = f"../images_georef/{index}.tif"

    # Save image as GeoTIFF
    with rasterio.open(
        output_path, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,  # RGB
        dtype='uint8',
        crs=CRS.from_epsg(4326),
        transform=transform
    ) as dst:
        img_data = img.convert("RGB")
        r, g, b = img_data.split()
        dst.write(np.array(r), 1)
        dst.write(np.array(g), 2)
        dst.write(np.array(b), 3)

    print(f"Georeferenced: {output_path}")